In [1]:
from flask import Flask, render_template, flash, request, redirect, url_for, session, logging
from flask_mysqldb import MySQL
from wtforms import Form, StringField, TextAreaField, PasswordField, validators
from passlib.hash import sha256_crypt
from functools import wraps
#from data import Articles
#Articles=Articles()

app = Flask(__name__)

#config MySQL
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER' ]= 'root'
app.config['MYSQL_PASSWORD'] = 'root'
app.config['MYSQL_DB'] ='myflaskapp'
app.config['MYSQL_CURSORCLASS'] = 'DictCursor'

#init MySQL
mysql=MySQL(app)

@app.route('/')
def helo():
    return render_template('home.html')

#About
@app.route('/about')
def about():
    return render_template('about.html')

#Articles 
@app.route('/articles')
def articles():
    cur = mysql.connection.cursor()
    # Get articles
    result = cur.execute("SELECT * FROM articles")
    articles = cur.fetchall()
    if result > 0:
        return render_template('articles.html', articles=articles)
    else:
        msg = 'No Articles Found'   
        return render_template('articles.html',msg=msg)  
    cur.close()

#Single Article
@app.route('/article/<string:id>/')
def article(id):
    cur = mysql.connection.cursor()
    result = cur.execute("SELECT * FROM articles WHERE id=%s",[id])
    article = cur.fetchone()
    return render_template('article.html', article=article  )

#Registrer form class
class RegisterForm(Form):
    name = StringField('Name',[validators.Length(min=1, max=25)])
    username = StringField('Username',[validators.Length(min=4, max=25)])
    email = StringField('Email',[validators.Length(min=6, max=25)])
    password = PasswordField('Password', [
        validators.DataRequired(),
        validators.EqualTo('confirm',message='password do not match')
    ])
    confirm = PasswordField('Confirm Password')

### ETAPE 2: l'utilisation de la base des données avec connecteur MYSQL_DB



#User Register
@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm(request.form)
    if request.method == 'POST' and form.validate():
        name=form.name.data
        email=form.email.data
        username=form.username.data
        password= sha256_crypt.encrypt(str(form.password.data))
        
        #create cursor
        cur = mysql.connection.cursor()
        
        #Execute query
        cur.execute("INSERT INTO users (name, email, username, password) VALUES(%s, %s, %s, %s)",(name, email, username, password))
        
        #commit to DB
        mysql.connection.commit()
        
        #close connection
        cur.close()
        
        flash('You are new regestred and can log in','success')
                
        return redirect(url_for('helo'))
         
        return render_template('register.html')        
    return render_template('register.html', form=form)
#####################################################################################################################################
@app.route('/login', methods=['GET','POST'])
def login():
    if request.method == 'POST':
        
        #Get form fields
        username = request.form['username']
        password_candidate = request.form['password']
        
        #Create cursor
        cur = mysql.connection.cursor()
        
        #Get user buy username
        result = cur.execute('SELECT * FROM users WHERE username = %s',[username])
        
        if result >0:
            #Get started hach
            data=cur.fetchone()
            password = data['password']
            
            #Compare Passwords
            if sha256_crypt.verify(password_candidate, password):
                #Passed
                session['logged_in'] = True
                session['username'] = username

                flash('You are now logged in','success')
                return redirect(url_for('dashboard'))
            else:
                error = 'Invalid login'
                return render_template('login.html',error=error)
            #Close connection
            cur.close()

        else:
            error = 'username not found'
            return render_template('login.html',error=error)   
            
    return render_template('login.html') 

#Check if user logged in
def is_logged_in(f):
    @wraps(f)
    def wrap(*args,**kwargs):
        if 'logged_in' in session:
            return f(*args, **kwargs)
        else:
            flash('Unauthorized, Please login','danger')
            return redirect(url_for('login')) 
    return wrap           

#Logout
@app.route('/logout')
def logout():
    session.clear()
    flash('You are now logged out','success')
    return redirect(url_for('login'))

#Dashboard
@app.route('/dashboard')
@is_logged_in
def dashboard():
    cur = mysql.connection.cursor()

    # Get articles
    result = cur.execute("SELECT * FROM articles")
    articles = cur.fetchall()
    if result > 0:
        return render_template('dashboard.html', articles=articles)
    else:
        msg = 'No Articles Found'   
        return render_template('dashboard.html', msg=msg)  
    cur.close()            

#Article Form Class
class ArticleForm(Form):
    title = StringField('Title',[validators.Length(min=1, max=250)])
    body = TextAreaField('Body',[validators.Length(min=15)])

#Add Article
@app.route('/add_article', methods=['GET','POST'])
@is_logged_in
def add_article():
    form = ArticleForm(request.form)
    if request.method == 'POST' and form.validate():
        title = form.title.data
        body = form.body.data
    
        #Create Cursor
        cur = mysql.connection.cursor()

        #execute
        cur.execute("INSERT INTO articles(title, body, author) VALUES(%s, %s, %s)",(title, body, session['username']))

        #Commit to DB
        mysql.connection.commit()

        #Close connection
        cur.close()
        flash('Article Created','success')
        
        return redirect(url_for('dashboard'))
  
    return render_template('add_article.html',form=form)

if __name__=='__main__' : 
    app.secret_key = 'super secret key'
    app.run()
    
# J'ai arrivé à 11:30 mn ds vid 4
#J'ai arrivé à 21:55 ds vid 4, ms il y a un problème ds l'affichege d'article et ds la nav "articles"


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
